In [1]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
!apt-get install -y -qq coinor-cbc

     |████████████████████████████████| 9.1 MB 15.5 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 148486 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up

In [2]:
from pyomo.environ import *
import numpy as np



In [3]:
m1 = ConcreteModel()
cons_rhs = np.array([10,-3,-6])
n = 3
m = 3

cons_coef = np.array([[2,1,2],[-1,1,2],[2,-3,1]])
obj_coefs = [1,-2,3]
varlb = np.array([1,2,1])
varub =np.array([np.inf,np.inf,np.inf])

rows = np.arange(n)
cols=np.arange(m)

In [4]:
m1.cons = ConstraintList()
m1.x = Var(cols)
for i in rows:
  m1.cons.add(sum(cons_coef[i][j]*m1.x[j] for j in cols) <= cons_rhs[i])

for i in cols:
  m1.x[i].setub(varub[i])
  m1.x[i].setlb(varlb[i])  


In [5]:
m1.cost = Objective(expr = summation(obj_coefs,m1.x), sense = minimize)

**Q1)**

In [6]:
opt = SolverFactory('glpk', executable='/usr/bin/glpsol').solve(m1)
#opt.write()
print('Solver status:',opt.solver.status)
print('Termination condition:',opt.solver.termination_condition)


Solver status: ok
Termination condition: other


Solver status ok suggests that the program was terminated normally\
Termination condition other means that an optimal solution was not found, but it does not specify the reason there was no solution.

In [7]:
optcbc =  SolverFactory('cbc').solve(m1)
#optcbc.write()
print('Solver status:',optcbc.solver.status)
print('Termination condition:',optcbc.solver.termination_condition)


    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>
Solver status: warning
Termination condition: infeasible


Solver status warning suggests that the problem could not be solved\
Termination condition infeasible means that an optimal solution was not found,since there was no solution that satisfied all constraints


For cbc solver the solver status is not completely meaningful, since it doesn't tell us what the warning is for.

### **Q2**

**A)**

In [8]:
m1a = m1.clone()

m1a.cost.set_sense(maximize)


In [9]:
cbca =  SolverFactory('cbc').solve(m1a)
#cbca.write()
print('Solver status:',cbca.solver.status)
print('Termination condition:',cbca.solver.termination_condition)

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>
Solver status: warning
Termination condition: infeasible


Solver status warning suggests that the problem could not be solved\
Termination condition infeasible means that an optimal solution was not found,since there was no solution that satisfied all constraints

**B**)

In [10]:
m1b = m1.clone()

In [11]:
m1b.x[2].setub(8)

In [12]:
cbcb = SolverFactory('cbc').solve(m1b)
#cbcb.write()
print('Solver status:',cbcb.solver.status)
print('Termination condition:',cbcb.solver.termination_condition)

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>
Solver status: warning
Termination condition: infeasible


Solver status warning suggests that the problem could not be solved\
Termination condition infeasible means that an optimal solution was not found,since there was no solution that satisfied all constraints

**C)**

In [13]:
m1c = m1.clone()

m1c.cons.add(-m1c.x[1]+m1c.x[2]<=9)


In [14]:
cbcc = SolverFactory('cbc').solve(m1c)
#cbcc.write()
print('Solver status:',cbcc.solver.status)
print('Termination condition:',cbcc.solver.termination_condition)

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>
Solver status: warning
Termination condition: infeasible


Solver status warning suggests that the problem could not be solved\
Termination condition infeasible means that an optimal solution was not found,since there was no solution that satisfied all constraints

**D)**

In [15]:
m1d = m1.clone()

m1d.cons.add(m1d.x[0]+m1d.x[1]>=45)
m1d.cons[1].deactivate()
m1d.cost2 = Objective(expr=m1d.x[0]+m1d.x[1]+m1d.x[2])
m1d.cost.deactivate()

In [16]:
cbcd = SolverFactory('cbc').solve(m1d)
print('Solver status:',cbcd.solver.status)
print('Termination condition:',cbcd.solver.termination_condition)

Solver status: ok
Termination condition: optimal


Status 'OK' suggests that the program terminated normally\
Termination condition optimal means that the program terminated because an optimal solution was found

In [17]:
print('Value of objective function is :', m1d.cost2())
for i in range(n):
  print('the value of x%d is'%(i+1), m1d.x[i].value)

Value of objective function is : 46.0
the value of x1 is 25.6
the value of x2 is 19.4
the value of x3 is 1.0


In [18]:
m1d.cons.display()

cons : Size=4
    Key : Lower : Body               : Upper
      2 :  None : -4.200000000000003 :  -3.0
      3 :  None : -5.999999999999993 :  -6.0
      4 :  45.0 :               45.0 :  None


In [19]:
print('The 2nd and 3rd constraints are active.',end=' ')
print('(Constraints with keys 3 and 4 respectively)')

The 2nd and 3rd constraints are active. (Constraints with keys 3 and 4 respectively)
